In [1]:
import tensorflow
import keras
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, Flatten, GlobalMaxPooling2D, Reshape
from tensorflow.keras.models import Sequential

In [2]:
train_path="Y:\College\Technical Seminar\Iris Dataset\Combined"

In [3]:
IMG_SHAPE = (224,224)
batch_size = 32

In [4]:
train_ds = tensorflow.keras.utils.image_dataset_from_directory(directory = train_path,
                                                               validation_split=0.2,
                                                               subset='training',
                                                               seed=123,
                                                       labels = "inferred",
                                                       label_mode = "categorical",
                                                       batch_size = batch_size,
                                                       image_size = IMG_SHAPE)

Found 6453 files belonging to 4 classes.
Using 5163 files for training.


In [5]:
val_ds=tensorflow.keras.preprocessing.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset='validation',
    seed=123,
    label_mode='categorical',
    image_size=IMG_SHAPE,
    batch_size=batch_size
)

Found 6453 files belonging to 4 classes.
Using 1290 files for validation.


In [21]:
densenet_model=Sequential()
pretrained_model=keras.applications.DenseNet121(include_top=False,
    input_shape=(224,224,3),
    pooling='avg',
    classes=4,
    weights='imagenet')

In [22]:
for layer in pretrained_model.layers:
    layer.trainable = False

for layer in pretrained_model.layers[-16:]:
    layer.trainable = True


In [23]:
densenet_model.add(pretrained_model)
densenet_model.add(Flatten())
densenet_model.add(Dense(512, activation='relu'))
densenet_model.add(Dense(256, activation='relu'))
densenet_model.add(Dense(4, activation='softmax'))

In [24]:
densenet_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1024)              7037504   
                                                                 
 flatten_2 (Flatten)         (None, 1024)              0         
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 7694660 (29.35 MB)
Trainable params: 985284 (3.76 MB)
Non-trainable params: 6709376 (25.59 MB)
_________________________________________________________________


In [25]:
densenet_model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(10e-4), metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall()])

In [26]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define early stopping and learning rate reduction callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, min_delta=1e-4)


In [27]:
history = densenet_model.fit(train_ds,
                    epochs=15,
                    validation_data=val_ds,
                    callbacks=[early_stop, reduce_lr],
                    verbose=1)

Epoch 1/15
162/162 [==============================] - 665s 4s/step - loss: 0.6903 - accuracy: 0.6744 - precision: 0.7583 - recall: 0.5942 - val_loss: 0.5970 - val_accuracy: 0.7078 - val_precision: 0.7181 - val_recall: 0.6930 - lr: 0.0010
Epoch 2/15
162/162 [==============================] - 646s 4s/step - loss: 0.3905 - accuracy: 0.8038 - precision: 0.8225 - recall: 0.7809 - val_loss: 0.4466 - val_accuracy: 0.7845 - val_precision: 0.7881 - val_recall: 0.7814 - lr: 0.0010
Epoch 3/15
162/162 [==============================] - 648s 4s/step - loss: 0.3084 - accuracy: 0.8536 - precision: 0.8632 - recall: 0.8433 - val_loss: 0.4401 - val_accuracy: 0.8194 - val_precision: 0.8269 - val_recall: 0.8147 - lr: 0.0010
Epoch 4/15
162/162 [==============================] - 640s 4s/step - loss: 0.2681 - accuracy: 0.8786 - precision: 0.8840 - recall: 0.8720 - val_loss: 0.3250 - val_accuracy: 0.8775 - val_precision: 0.8794 - val_recall: 0.8760 - lr: 0.0010
Epoch 5/15
162/162 [============================

In [28]:
densenet_model.save("DenseNet_Combined.h5")

C:\Users\Yash\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
import cv2
import numpy as np
import pandas as pd

In [14]:
img_dir = "E:\\Dataset\\Clarkson_LG\\Live"
import os
img_files = os.listdir(img_dir)
features_list = []

In [15]:
for img_file in img_files:
    # Load the image and preprocess it
    img_path = os.path.join(img_dir, img_file)
    image=cv2.imread(img_path)
    image_resized=cv2.resize(image,(224,224))
    image=np.expand_dims(image_resized,axis=0)
    features=densenet_model.layers[-4].output
    feature=keras.models.Model(inputs=densenet_model.input,outputs=features).predict(image)
    features_list.append(feature)

1/1 [==============================] - 2s 2s/step


In [16]:
features_array = np.vstack(features_list)
df = pd.DataFrame(features_array)
num_features = features_array.shape[1]
col_names = ['feature{}'.format(i) for i in range(1, num_features+1)]
df.columns = col_names
df['filename'] = img_files
df.set_index('filename', inplace=True)
excel_file = "E:\\2ndPaper\\DCNN Features\\LG_Live.xlsx"
df.to_excel(excel_file)

In [17]:
img_dir = "E:\\Dataset\\Clarkson_LG\\Pattern"
import os
img_files = os.listdir(img_dir)
features_list = []

In [18]:
for img_file in img_files:
    # Load the image and preprocess it
    img_path = os.path.join(img_dir, img_file)
    image=cv2.imread(img_path)
    image_resized=cv2.resize(image,(224,224))
    image=np.expand_dims(image_resized,axis=0)
    features=densenet_model.layers[-4].output
    feature=keras.models.Model(inputs=densenet_model.input,outputs=features).predict(image)
    features_list.append(feature)

1/1 [==============================] - 2s 2s/step


In [19]:
features_array = np.vstack(features_list)
df = pd.DataFrame(features_array)
num_features = features_array.shape[1]
col_names = ['feature{}'.format(i) for i in range(1, num_features+1)]
df.columns = col_names
df['filename'] = img_files
df.set_index('filename', inplace=True)
excel_file = "E:\\2ndPaper\\DCNN Features\\LG_Pattern.xlsx"
df.to_excel(excel_file)

In [20]:
img_dir = "E:\\Dataset\\Clarkson_LG\\Printed"
import os
img_files = os.listdir(img_dir)
features_list = []

In [21]:
for img_file in img_files:
    # Load the image and preprocess it
    img_path = os.path.join(img_dir, img_file)
    image=cv2.imread(img_path)
    image_resized=cv2.resize(image,(224,224))
    image=np.expand_dims(image_resized,axis=0)
    features=densenet_model.layers[-4].output
    feature=keras.models.Model(inputs=densenet_model.input,outputs=features).predict(image)
    features_list.append(feature)

1/1 [==============================] - 4s 4s/step


In [22]:
features_array = np.vstack(features_list)
df = pd.DataFrame(features_array)
num_features = features_array.shape[1]
col_names = ['feature{}'.format(i) for i in range(1, num_features+1)]
df.columns = col_names
df['filename'] = img_files
df.set_index('filename', inplace=True)
excel_file = "E:\\2ndPaper\\DCNN Features\\LG_Printed.xlsx"
df.to_excel(excel_file)